In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=29758352f593ec5c99451acf594606758cefa80a32cc4e3acde34c0e67dfaeea
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
from pyspark import SparkContext, SparkConf

In [ ]:
conf = SparkConf().setMaster("local").setAppName("Wiki")

In [ ]:
sc = SparkContext(conf=conf)


In [ ]:

rdd = sc.textFile("/content/pagecounts-20160101-000000_parsed.out")
count=rdd.count()
print(count)

3324129


Q1 with for loops

In [ ]:
import numpy as np
import time
start_time = time.time()
# create an empty list to store page sizes
page_sizes = []

# iterate over each line in the RDD
for line in rdd.toLocalIterator():
    # extract the page size from the line
    if len(line.split(" ")) != 1:
      _page_size = line.split(" ")[-1]
      if _page_size == "null":
        page_sizes.append(int(0))
      else:
        page_sizes.append(int(_page_size))
    else:
      continue

# create a new RDD from the page_sizes list


# find the minimum, maximum, and average page sizes
min_page_title = float('inf')
max_page_title = float('-inf')
sum_page_title = 0
count = 0

for value in page_sizes:
    # Update minimum value
    if value < min_page_title:
        min_page_title = value

    # Update maximum value
    if value > max_page_title:
        max_page_title = value

    # Accumulate sum
    sum_page_title += value
    count += 1

avg_page_title = sum_page_title / count

print("Minimum page title size:", min_page_title)
print("Maximum page title size:", max_page_title)
print("Average page title size:", avg_page_title)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")



Minimum page title size: 0
Maximum page title size: 141180155987
Average page title size: 132239.56957446598
Elapsed time: 24.491726636886597 seconds


Q1 with mapreduce

In [ ]:
import time
start_time = time.time()
if len(line.split(" ")) != 1:
  page_size_rdd = rdd.map(lambda line: int(line.split(" ")[-1]) if line.split(" ")[-1] != "null" else 0)
  min_page_title = page_size_rdd.min()


  max_page_title = page_size_rdd.max()


  avg_page_title = page_size_rdd.mean()

avg_page_title
writalbe_rdd = sc.parallelize([min_page_title, max_page_title, avg_page_title])

writalbe_rdd.saveAsTextFile('./q111')
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")

Elapsed time: 32.142927169799805 seconds


Q2 with for loops

In [ ]:
import time
start_time = time.time()
# create an empty list to store results
results = []
count = 0
c=0
# loop through the RDD and apply the filter and count operations
for line in rdd.toLocalIterator():
    # split the line into language code and page title
    if len(line.split(" ")) >= 2:
      code, title = line.split(" ")[0], line.split(" ")[1]
    # check if the page title starts with "The" and the language code is not "en"
      if (code !='en' and  title[0:4] == 'The_'):
        # print("Code: ",code," , Title: ", title)
        count= count + 1
      if ( title[0:4] == 'The_'):
        # print("Code: ",code," , Title: ", title)
        c= c + 1

# get the count of pages
print(count)
print(c)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")

9160
41901
Elapsed time: 33.4593026638031 seconds


Q2 with mapreduce


In [ ]:
import time
start_time = time.time()
if len(line.split(" ")) >= 2:
  page_title_with_code_rdd = rdd.map(lambda line: int(line.split(" ")[-1]) if line.split(" ")[-1] != "null" else 0)

  page_title_with_the = rdd.map(lambda line: (line.split(" ")[0], line.split(" ")[1])) \
.filter(lambda line: line[1][0:4] == 'The_' and line[0] != 'en')
  page_title_with_the_ = rdd.map(lambda line: (line.split(" ")[0], line.split(" ")[1])) \
.filter(lambda line: line[1][0:4] == 'The_' )

  count = page_title_with_the.count()
  c= page_title_with_the_.count()

writalbe_rdd = sc.parallelize([count,c])

writalbe_rdd.saveAsTextFile('./q24')
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")

Elapsed time: 11.301353693008423 seconds


Q3 with mapreduce


In [ ]:
import time
start_time = time.time()
page_title_rdd = rdd.map(lambda line: line.split(" ")[1] if len(line.split(" ")) >= 2 else line.split(" ")[0])


splited_title = page_title_rdd.flatMap(lambda line: line.split('_'))


normalized_splited_title = splited_title.map(lambda line: line.lower())
normalized_splited_title = normalized_splited_title.map(lambda line: ''.join(ch for ch in line if ch.isalnum()))

from operator import add
term_count = normalized_splited_title.map(lambda term: (term,1)).reduceByKey(add)
distinct_terms = term_count.filter(lambda term: term[1] == 1)
count = distinct_terms.count()


writalbe_rdd = sc.parallelize([count])

writalbe_rdd.saveAsTextFile('./q3')
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")

Elapsed time: 29.787956476211548 seconds


Q3 with for loop


In [ ]:
# Determine the number of unique terms appearing in the page titles. Note that in page
# titles, terms are delimited by “_” instead of a white space. You can use any number of
# normalization steps (e.g. lowercasing, removal of non-alphanumeric characters).

# get page titles
import time
start_time = time.time()
title_list = []
for line in page_title_rdd.toLocalIterator():
  if len(line.split(" ")) >= 2:
    title = line.split(" ")[1]
    title_list.append(title)

# split titles into words

splited_title = []
for line in page_title_rdd.toLocalIterator():
    words = line.split('_')
    splited_title.extend(words)

normalized_terms = []
for term in splited_title:
    term_normalized = term.lower()
    term_normalized = ''.join(ch for ch in term_normalized if ch.isalnum())
    normalized_terms.append(term_normalized)


# normalize terms

# count distinct terms
term_count = {}
for term in normalized_splited_title.toLocalIterator():
    if term not in term_count:
        term_count[term] = 1
    else:
        term_count[term] += 1

distinct_terms = sc.parallelize([(term, count) for term, count in term_count.items() if count == 1])

# get count of distinct terms
count = 0
for term in distinct_terms.toLocalIterator():
    count += 1
print(count)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")

1292290
Elapsed time: 75.46503973007202 seconds


Q4 with mapreduce

In [ ]:
import time
start_time = time.time()
page_title_rdd = rdd.map(lambda line: line.split(" ")[1] if len(line.split(" ")) >= 2 else line.split(" ")[0])
from operator import add
# Count the frequency of each title
freq_title = page_title_rdd.map(lambda term: (term,1)).reduceByKey(lambda a,b:a +b)
most_freq_title = freq_title.sortBy(lambda pair: pair[1], False)

# Print the results using foreach
most_freq_title.foreach(lambda x: print(f"({x[0]}, {x[1]})"))


most_freq_title.filter(lambda x: x[0]).saveAsTextFile('./q4544451')
# most_freq = most_freq_title.first()

end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")


Elapsed time: 72.33464050292969 seconds


Q4 with for loops


In [ ]:
import time
start_time = time.time()

# Count the frequency of each title
title_counts = {}
for line in rdd.toLocalIterator():
    fields = line.split(" ")
    title = fields[1] if len(fields) >= 2 else fields[0]
    if title not in title_counts:
        title_counts[title] = 1
    else:
        title_counts[title] += 1

# Sort the title counts in descending order
sorted_counts = sorted(title_counts.items(), key=lambda item: item[1], reverse=True)

# Print the results
for title, count in sorted_counts:
    print(f"({title}, {count})")

# # Save the results to a file
# with open('./q45444512', 'w') as f:
#     for title, count in sorted_counts:
#         f.write(f"({title}, {count})\n")

end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")


Streaming output truncated to the last 5000 lines.
(Template:PRC_admin/data/44/15/81/001/002, 1)
(Template:PRC_admin/data/44/16/02/002/009, 1)
(Template:PRC_admin/data/44/16/22/115/210, 1)
(Template:PRC_admin/data/44/16/25/101/204, 1)
(Template:PRC_admin/data/44/16/25/102/205, 1)
(Template:PRC_admin/data/44/16/25/103/217, 1)
(Template:PRC_admin/data/44/17/23/105/209, 1)
(Template:PRC_admin/data/44/18/02/107/212, 1)
(Template:PRC_admin/data/44/18/26/109/206, 1)
(Template:PRC_admin/data/44/18/27/104/208, 1)
(Template:PRC_admin/data/44/18/27/113/202, 1)
(Template:PRC_admin/data/44/18/81/111/209, 1)
(Template:PRC_admin/data/44/19/00/003/017, 1)
(Template:PRC_admin/data/44/51/02/005/005, 1)
(Template:PRC_admin/data/44/51/21/109/202, 1)
(Template:PRC_admin/data/44/51/21/109/203, 1)
(Template:PRC_admin/data/44/51/22/100/225, 1)
(Template:PRC_admin/data/44/52/02/001/003, 1)
(Template:PRC_admin/data/44/52/02/001/008, 1)
(Template:PRC_admin/data/44/52/02/006/212, 1)
(Template:PRC_admin/data/44/5

Q5 from mapreduce

In [ ]:
# Assuming 'lines' is an RDD containing input data
start_time = time.time()
combined_data = rdd.map(lambda line: (line.split(" "),)) \
    .map(lambda fields: (fields[0][1], (fields[0][2:-1], [fields[0][-1]]))) \
    .reduceByKey(lambda acc, value: (acc[0] + value[0], acc[1] + value[1])) \
    .mapValues(lambda x: {"hinst": ",".join(x[0]), "size": x[1]})

# Save the output to a file
combined_data.map(lambda x: f"(Title: {x[0]}\nPage data:\n    Hinst: {x[1]['hinst']}\n    Size: {x[1]['size']})\n\n") \
    .saveAsTextFile("output72_mapreduce.txt")
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")


Elapsed time: 79.51406145095825 seconds


# Q5 with for loops

In [ ]:
import time

start_time = time.time()
parsed_data = []
for line in rdd.toLocalIterator():
    fields = line.split(" ")
    title = fields[1]
    hinst = fields[2:-1]  # Extract hinst values
    size = fields[-1]  # Extract size value
    parsed_data.append((title, (hinst, [size])))

combined_data = {}
for title, data in parsed_data:
    hinst, size = data
    if title not in combined_data:
        combined_data[title] = ([], [])
    combined_data[title][0].extend(hinst)
    combined_data[title][1].extend(size)

result = {}
for title, data in combined_data.items():
    hinst = data[0]
    size = data[1]
    result[title] = {"hinst": hinst, "size": size}



for title, data in result.items():
    hinst = ", ".join(data["hinst"])
    size = data["size"]
    print(f"(Title: {title} , Page data:  Hinst: {hinst}  Size: {size} )\n\n")
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")


Streaming output truncated to the last 5000 lines.

(Title: Bielsk_(gmina) , Page data:  Hinst: 1  Size: ['8984'] )


(Title: Billey , Page data:  Hinst: 1  Size: ['9328'] )


(Title: Bordeaux-Saint-Clair , Page data:  Hinst: 1  Size: ['8911'] )


(Title: Brestot , Page data:  Hinst: 1  Size: ['8949'] )


(Title: Bricherasio , Page data:  Hinst: 1  Size: ['9459'] )


(Title: Buxeuil_(Indre) , Page data:  Hinst: 1  Size: ['9097'] )


(Title: Calmels-et-le-Viala , Page data:  Hinst: 1  Size: ['9117'] )


(Title: Chauchailles , Page data:  Hinst: 1  Size: ['8937'] )


(Title: Chheh , Page data:  Hinst: 1  Size: ['14971'] )


(Title: City_Hunter_(SBS) , Page data:  Hinst: 1  Size: ['9263'] )


(Title: Clavette , Page data:  Hinst: 1  Size: ['9128'] )


(Title: Coca_(Segovia) , Page data:  Hinst: 1  Size: ['8881'] )


(Title: Colin_Graeme_Roe , Page data:  Hinst: 1  Size: ['5508'] )


(Title: Consolation-Maisonnettes , Page data:  Hinst: 1  Size: ['9016'] )


(Title: Coulandon , Page data: 